In [ ]:
# import dependencies
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.io import loadmat, wavfile
import random

import os
from pathlib import Path

import tensorflow as tf
from tensorflow import keras

In [ ]:
# load data
data = []
pathlist = Path('DataIn').glob('*.wav')
for p in pathlist:
  fs, x = wavfile.read(str(p))
  fs, y = wavfile.read('Data100/'+p.name)
  data.append([x, y])

# shuffle data
random.shuffle(data)

In [ ]:
# separate x and y data
x_values = []
y_values = []

for d in data:
    x_values.append(d[0][14000:])
    y_values.append(d[1][14000:])

x_values = np.asarray(x_values) / 2**15
y_values = (np.asarray(y_values) / 2**15) * 2
print(np.shape(x_values))
print(np.shape(y_values))

In [ ]:
# plot sample data point
idx = 10
plt.plot(x_values[idx])
plt.plot(y_values[idx])
# plt.xlim(2000, 3000)
plt.show()

In [ ]:
# use float32 data type...
x_values = x_values.astype(np.float32)
y_values = y_values.astype(np.float32)

In [ ]:
# split into training and validation data
NUM_TRAIN = 400
NUM_VAL = 25
x_train, x_val = np.split(x_values, [NUM_TRAIN])
y_train, y_val = np.split(y_values, [NUM_TRAIN])

print(np.shape(x_train))
print(np.shape(x_val))
print(np.shape(y_train))

In [ ]:
# reshape data
NUM_SAMPLES=8050
IN_train    = np.reshape(x_train, (NUM_TRAIN, NUM_SAMPLES, 1))
IN_validate = np.reshape(x_val,   (NUM_VAL,   NUM_SAMPLES, 1))

OUT_train    = np.reshape(y_train, (NUM_TRAIN, NUM_SAMPLES, 1))
OUT_validate = np.reshape(y_val,   (NUM_VAL,   NUM_SAMPLES, 1))

print(np.shape(IN_train))

In [ ]:
model = tf.keras.Sequential()

# GRU Layer
model.add(keras.layers.GRU(units=8, input_shape=(NUM_SAMPLES,1), return_sequences=True, bias_initializer='random_uniform'))

# Final layer is 1 neuron -> single value output
model.add(keras.layers.Dense(1))

# Adam optimizer
opt = keras.optimizers.Adam(learning_rate=2.0e-3)

# Error-to-Signal ratio loss function
def esr_loss(target_y, predicted_y):
  return tf.reduce_sum(tf.square(target_y - predicted_y)) / tf.reduce_sum(tf.square(target_y))

model.compile(optimizer=opt, loss=esr_loss, metrics=['mse'])
model.summary()

In [ ]:
history = model.fit(IN_train, OUT_train, epochs=1000, validation_data=(IN_validate, OUT_validate))

In [ ]:
# plot loss
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# plot error
plt.clf()
mse = history.history['mse']
val_mse = history.history['val_mse']

plt.plot(epochs, mse, 'g.', label='Training MSE')
plt.plot(epochs, val_mse, 'b.', label='Validation MSE')
plt.title('Training and validation mean square error')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
# Test prediction
idx = 3
predictions = model.predict(x_train[idx].reshape(1, NUM_SAMPLES, 1))
print(np.shape(predictions))

# Plot the predictions along with to the test data
plt.clf()
plt.title('Training data predicted vs actual values')
plt.plot(y_train[idx], 'b', label='Actual')
plt.plot(predictions.flatten(), 'r--', label='Predicted')
plt.legend()
plt.xlim(2000, 3000)
plt.xlabel('Time [samples]')
plt.ylabel('Voltage [V]')
plt.show()

In [ ]:
# Test prediction
def freqSmooth(x, sm=1.0/24.0):
    s = sm if sm > 1.0 else np.sqrt(2.0**sm)
    N = len(x)
    y = np.zeros_like(x)
    for i in range(N):
        i1 = max(int(np.floor(i/s)), 0)
        i2 = min(int(np.floor(i*s)+1), N-1)
        if i2 > i1:
            y[i] = np.mean(x[i1:i2])
    return y

idx = 3
predictions = model.predict(x_train[idx].reshape(1, NUM_SAMPLES, 1))
print(np.shape(predictions))

pred_fft = freqSmooth(20 * np.log10(np.abs(np.fft.rfft(predictions.flatten()))))
target_fft = freqSmooth(20 * np.log10(np.abs(np.fft.rfft(y_train[idx]))))
freqs = np.fft.rfftfreq(NUM_SAMPLES, 1.0 / 44100.0)

# Plot the predictions along with to the test data
plt.clf()
plt.title('Training data predicted vs actual values')
plt.semilogx(freqs, target_fft, 'b', label='Actual')
plt.semilogx(freqs, pred_fft, 'r--', label='Predicted')
plt.legend()
plt.xlim(50, 20000)
plt.ylim(-5)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude [dB]')
plt.show()

In [ ]:
# Export weights....
print('GRU Kernel Weights...')
print(model.weights[0].numpy())

print('GRU Recurrent Weights...')
print(model.weights[1].numpy())

print('GRU bias...')
print(model.weights[2].numpy())

print('Dense Kernel Weights...')
print(model.weights[3].numpy())

print('Dense bias...')
print(model.weights[4].numpy())